In [14]:
import argparse
from pathlib import Path
import pickle
import mlflow

In [15]:
model_name = "taxi-model"
model_path = "/tmp/train"
evaluation_output = "/tmp/evaluate"

In [16]:
def main(model_name, model_path, evaluation_output):
    '''Loads model, registers it if deply flag is True'''

    with open((Path(evaluation_output) / "deploy_flag"), 'rb') as infile:
        deploy_flag = int(infile.read())
    
    mlflow.log_metric("deploy flag", int(deploy_flag))
    if deploy_flag==1:

        print("Registering ", model_name)

        # load model
        with open((Path(model_path) / "model.pkl"), "rb") as infile:
            model = pickle.load(infile)

        # log model using mlflow
        mlflow.sklearn.log_model(model, model_name)

        # register logged model using mlflow
        run_id = mlflow.active_run().info.run_id
        model_uri = f'runs:/{run_id}/{model_name}'
        mlflow.register_model(model_uri, model_name)

    else:
        print("Model will not be registered!")

In [17]:
mlflow.start_run()

lines = [
    f"Model path: {model_path}",
    f"Evaluation output path: {evaluation_output}",
]

for line in lines:
    print(line)

main(model_name, model_path, evaluation_output)

mlflow.end_run()

Model path: /tmp/train
Evaluation output path: /tmp/evaluate
Model will not be registered!
